In [0]:
import cats._, cats.data._, cats.implicits._
import org.scalacheck.Prop

In [1]:
1 == "1"

false

In [2]:
val l = List(1, 2, 3, 4)
Functor[List].map(l)(_ + 1)

List(2, 3, 4, 5)

In [3]:
val l = List(1, 2) |+| List(3, 4, 5)
l

List(1, 2, 3, 4, 5)

In [4]:
val l1: List[Any] = List(1, 2, 3) 
val l2: List[Any] = List("4", "5")
l1 |+| l2

List(1, 2, 3, 4, 5)

In [5]:
val a: Either[String, Int] = Right(1)
a.map(_ + 1)

Right(2)

In [6]:
val f = ((x: Int) => x + 1).compose((_: Int) * 7)
f(1)

8

In [7]:
val f = ((x: Int) => x + 1).map((_: Int) * 7) 
f(1)

14

**Type Class** - API representing some functionality. In scala it's a trait with type parameter.<br>**Type Class** instance provides implementations for the type.

**Type Class Interface** - functionality exposed to the used. This can be **interface object** (methods in singleton object) or **interface syntax** (extension methods).

 



In [9]:
trait Printable[A] {
    def format[A](value: A): String
}
object PrintableInstances {
    implicit val printableString: Printable[String] = new Printable[String]{
        def format[String](value: String) = value.toString
    }
    implicit val printableInt: Printable[Int] = new Printable[Int]{
        def format[Int](value: Int) = value.toString
    }
}
object Printable{
    def format[A](value: A)(implicit valuePrintable: Printable[A]): String = valuePrintable.format(value)
    def print[A](value: A)(implicit valuePrintable: Printable[A]): Unit = println(valuePrintable.format(value))
}

import PrintableInstances._, Printable._
format(3)

3

In [10]:
object PrintableInstances {
    implicit val printableString: Printable[String] = new Printable[String]{
        def format[String](value: String) = value.toString
    }
    implicit val printableInt: Printable[Int] = new Printable[Int]{
        def format[Int](value: Int) = value.toString
    }
}
object PrintableSyntax{
    implicit class PrintableOps[A](value: A) {
        def format(implicit p: Printable[A]): String = p.format(value)
        def print(implicit p: Printable[A]): Unit = println(p.format(value))
    }
}

import PrintableInstances._, PrintableSyntax._
3.print // works in intellij

3


In [11]:
case class En(x: Int, y: String) 
implicit val enShow: Show[En] = Show.show[En](en => s"${en.x} ${en.y}")
val en = En(1, "2")
en.show

1 2

In [12]:
val l1 = List(Some(1), None, Some(2))
l1.reduce((e1, e2) => e1 |+| e2)

Some(3)

In [13]:
// Monoid.apply[Boolean].empty // does not exist

In [14]:
trait Codec[A] { self => 
  def encode(value: A): String
  def decode(value: String): A
  def imap[B](dec: A => B, enc: B => A): Codec[B] = {
      new Codec[B]{
          def encode(value: B): String = {
              self.encode(enc(value))
          }
          def decode(value: String): B = {
              dec(self.decode(value))
          }
      }
  }
}
val doubleCodec = new Codec[Double]{
    def encode(value: Double) = value.toString
    def decode(value: String) = value.toDouble
}

doubleCodec.decode("2")
doubleCodec.encode(2.0)


2.0

In [15]:
import cats.Id

def pure[A](value: A): Id[A] = value
def flatMap[A, B](value: Id[A])(f: A => Id[B]): Id[B] = f(value)
def map[A, B](value: Id[A])(f: A => B): Id[B] = f(value)

2

In [16]:
val r1: Either[String, Int] = Right(3)
val r2: Either[String, Int] = Left("s")
val r3 = Right(5)
for {
    vr1 <- r1.right
    vr2 <- r2.right
} yield vr1 * vr2
// TODO: Eval.defer

Left(s)